<a href="https://colab.research.google.com/github/Navjotkhatri/Productionization_of_ML_Systems_in_Travel_Industry/blob/main/REST_API_for_Flight_Price_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. REST API for Regression Model:**

**Develop a REST API using Flask to serve the flight price prediction model, enabling real-time price predictions.**

In [43]:
from flask import Flask


In [44]:
app = Flask(__name__)

In [45]:
!pip install flask_ngrok pyngrok sentence-transformers

In [46]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from pyngrok import ngrok
from datetime import datetime
from google.colab import drive
from flask_ngrok import run_with_ngrok
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template, request, jsonify
from sklearn.preprocessing import StandardScaler

In [47]:
import numpy as np
from sklearn.preprocessing import StandardScaler

def predict_price(input_data, model):
  text_columns=['from','to','flightType','agency','weekday_num','month','year','speed']
  df=pd.DataFrame([input_data])
  for column in text_columns:
    df[column+'_embedding']=df[column].apply(lambda text: model.encode(text))
  #X=scaler.transform(df)
  y_pred=model.predict(X)
  return y_pred[0]

In [48]:
import pickle

# Open the file in binary read mode ('rb')
with open('/content/flight_price_prediction_regression.pkl', 'rb') as file:
    model = pickle.load(file)  # Load the model from the file object

In [49]:
!ngrok authtoken "2kEiDvlDvDmlSrjFdidw1UnLQmx_32guShAgCSJ9W3gqsYXPA"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [50]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['GET', 'POST'])
def predict():
    return """
    <!-- Insert the HTML form code here -->
    <!DOCTYPE html>
<html>

<head>
    <title>Flight Price Prediction</title>
        <style>
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }

        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 40px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.1);
            text-align: center;
        }

        h1 {
            color: #184344;
            font-size: 36px;
            margin-bottom: 20px;
        }

        form {
            text-align: left;
        }

        input[type="text"],
        input[type="number"] {
            width: 100%;
            padding: 15px;
            margin: 15px 0;
            border: none;
            border-bottom: 2px solid #184344;
            font-size: 18px;
            background-color: transparent;
            color: #333;
            transition: border-bottom 0.3s ease;
        }

        input[type="text"]:focus,
        input[type="number"]:focus {
            border-bottom: 2px solid #184344;
            outline: none;
        }

        input[type="checkbox"] {
            margin-right: 10px;
        }

        input[type="submit"] {
            background-color: #184344;
            color: #fff;
            padding: 15px 30px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 20px;
            transition: background-color 0.3s ease;
        }

        input[type="submit"]:hover {
            background-color: #184344;
        }

        p#prediction {
            margin-top: 20px;
            font-size: 24px;
            color: #184344;
        }
    </style>
</head>

<body>
    <div class="container">
        <h1>Flight Price Prediction</h1>
        <form action="/predict" method="POST">

            <label>Departure City:</label><br><br>
            <input type="radio" name="from_city" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="radio" name="from_city" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="radio" name="from_city" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <!-- Other cities... -->
            <br>

            <label>Destination City:</label><br><br>
            <input type="radio" name="to_city" value="Aracaju (SE)">Aracaju (SE)<br>
            <input type="radio" name="to_city" value="Brasilia (DF)">Brasilia (DF)<br>
            <input type="radio" name="to_city" value="Campo Grande (MS)">Campo Grande (MS)<br>
            <!-- Other cities... -->
            <br>

            <label>Flight Type:</label><br><br>
            <input type="radio" name="flightType" value="economic"> Economic<br>
            <input type="radio" name="flightType" value="firstClass"> FirstClass<br>
            <input type="radio" name="flightType" value="premium"> Premium<br>
            <br>

            <label>Agency:</label><br><br>
            <input type="radio" name="agency" value="CloudFy"> CloudFy<br>
            <input type="radio" name="agency" value="FlyingDrops"> FlyingDrops<br>
            <input type="radio" name="agency" value="Rainbow"> Rainbow<br>
            <br>

            <label for="weekday_num">Weekday (0=Sunday, 6=Saturday):</label>
            <input type="number" name="weekday_num" min="0" max="6" placeholder="Day of the week"><br>

            <label for="month">Month:</label>
            <input type="number" name="month" min="1" max="12" placeholder="Month"><br>

            <label for="year">Year:</label>
            <input type="number" name="year" min="2023" max="2123" placeholder="Year"><br>

            <input type="submit" value="Predict">
        </form>
        <p id="prediction"></p>
    </div>
</body>

</html>

    """

@app.route('/predict', methods=['POST'])
def index():
    if request.method == 'POST':
        # Get input data from the form
        from_city = request.form.get('from_city')
        to_city = request.form.get('to_city')
        flighttype = request.form.get('flightType')
        agency = request.form.get('agency')
        weekday_num = request.form.get('weekday_num')
        month = request.form.get('month')
        year = request.form.get('year')

        # Default speed value if speed is not part of the form
        speed = 800  # Example default value, adjust as needed

        # Create a dictionary to store the input data
        data = {
            'From': from_city,
            'To': to_city,
            'flightType': flighttype,
            'agency': agency,
            'weekday_num': weekday_num,
            'month': month,
            'year': year,
            'speed': speed  # using default value
        }

        # Perform prediction using the input data dictionary
        predicted_price = str(round(predict_price(data, model), 2))

        return jsonify({'prediction': predicted_price})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll